In [1034]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import resample
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import minmax_scale

import tensorflow as tf
import keras

# Import various componenets for model building
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Flatten, Conv1D, MaxPooling1D
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers import Input, TimeDistributed
from keras.models import Model
from keras.optimizers import SGD

# Import the backend
from keras import backend as K
from datetime import datetime
%matplotlib inline

In [1035]:
main_start = datetime.now()
raw_data = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

raw_data.head(10)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
5,9305-CDSKC,Female,0,No,No,8,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes
6,1452-KIOVK,Male,0,No,Yes,22,Yes,Yes,Fiber optic,No,...,No,No,Yes,No,Month-to-month,Yes,Credit card (automatic),89.10,1949.4,No
7,6713-OKOMC,Female,0,No,No,10,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,No,Mailed check,29.75,301.9,No
8,7892-POOKP,Female,0,Yes,No,28,Yes,Yes,Fiber optic,No,...,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes
9,6388-TABGU,Male,0,No,Yes,62,Yes,No,DSL,Yes,...,No,No,No,No,One year,No,Bank transfer (automatic),56.15,3487.95,No


In [1036]:
# Let's take a look at the type of data in each column
# Drop the CustomerID since it's a unique key that means nothing to us.
data = raw_data.drop(['customerID'], axis=1)
data.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [1037]:
#It appears that Total Charges is coming in as a string, so let's make it numeric
data['TotalCharges'] = data['TotalCharges'].convert_objects(convert_numeric=True)

/Users/bryancorder/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


In [1038]:
# Let's check the data for nulls
null_count = data.isnull().sum()
null_count[null_count>0]

TotalCharges    11
dtype: int64

In [1039]:
# There appears to be 11 rows that are missing Total Charges. Since the number is insignificant, let's drop the rows.
data = data.dropna(axis=0)

In [1040]:
#For the columns that only have 2 values yes or no, let's convert them to 0s and 1s
#Hold off on converting Churn to 0s and 1s for graphs below
Yes_No_Columns = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']

for feature in Yes_No_Columns:
    data[feature] = np.where(data[feature].values == 'Yes', 1, 0)

#Now let's convert the Males/Females in 0 (Males) and 1 (Females)
data['gender'] = np.where(data.gender.values == 'Female', 1, 0)

#Let's see what we've created
data.head(10)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,1,0,1,0,1,0,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,1,Electronic check,29.85,29.85,No
1,0,0,0,0,34,1,No,DSL,Yes,No,Yes,No,No,No,One year,0,Mailed check,56.95,1889.50,No
2,0,0,0,0,2,1,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,1,Mailed check,53.85,108.15,Yes
3,0,0,0,0,45,0,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,0,Bank transfer (automatic),42.30,1840.75,No
4,1,0,0,0,2,1,No,Fiber optic,No,No,No,No,No,No,Month-to-month,1,Electronic check,70.70,151.65,Yes
5,1,0,0,0,8,1,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,1,Electronic check,99.65,820.50,Yes
6,0,0,0,1,22,1,Yes,Fiber optic,No,Yes,No,No,Yes,No,Month-to-month,1,Credit card (automatic),89.10,1949.40,No
7,1,0,0,0,10,0,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,0,Mailed check,29.75,301.90,No
8,1,0,1,0,28,1,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,1,Electronic check,104.80,3046.05,Yes
9,0,0,0,1,62,1,No,DSL,Yes,Yes,No,No,No,No,One year,0,Bank transfer (automatic),56.15,3487.95,No


In [1041]:
#Now convert Churn to 0s and 1s.
data['Churn'] = np.where(data['Churn'].values == 'Yes', 1, 0)
#Let's get dummies for the columns with multiple answers
multiple_answers = ['MultipleLines','InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                    'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract','PaymentMethod']

data = pd.get_dummies(data, columns=multiple_answers)

In [1042]:
data['tenure_norm'] = (data['tenure'] - min(data['tenure']))/ (max(data['tenure']) - min(data['tenure']))
data['MonthlyCharges_norm'] = (data['MonthlyCharges'] - min(data['MonthlyCharges']))/ (max(data['MonthlyCharges']) - min(data['MonthlyCharges']))
data['TotalCharges_norm'] = (data['TotalCharges'] - min(data['TotalCharges']))/ (max(data['TotalCharges']) - min(data['TotalCharges']))
model_data = data.drop(['TotalCharges', 'MonthlyCharges', 'tenure'], axis=1)

In [1043]:
# Create a separate yes and no variable
yes_churn = model_data[model_data.Churn == 1]
no_churn = model_data[model_data.Churn == 0]

# Use resample to get an equal number of yes and nos in our data
model_data = resample(no_churn, n_samples=1869)
model_data = model_data.append(yes_churn)

# Now we can create our X and Y variables
X = model_data.drop(['Churn'], 1)
y = model_data['Churn']

#Make a training and test set.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [1044]:
start = datetime.now()
#Use Lasso/RFE with Cross Validation to get the best features for our second X
lr = LogisticRegression(C=1, penalty='l1')

#Set up our X,y
X = X_train
y = y_train

#Set up our Recursive Feature Elimination
rfe = RFECV(lr,cv=5) 
fit = rfe.fit(X,y)
result_RFE = pd.DataFrame(list(zip(X.head(0), rfe.ranking_, rfe.support_)),
                          columns=['Features','Ranking','Support'] ) 
print(result_RFE.sort_values('Ranking'))
print('It took the following time to complete this task:', datetime.now() - start)

                                   Features  Ranking  Support
0                                    gender        1     True
37                              tenure_norm        1     True
35           PaymentMethod_Electronic check        1     True
33  PaymentMethod_Bank transfer (automatic)        1     True
32                        Contract_Two year        1     True
31                        Contract_One year        1     True
30                  Contract_Month-to-month        1     True
29                      StreamingMovies_Yes        1     True
26                          StreamingTV_Yes        1     True
23                          TechSupport_Yes        1     True
21                           TechSupport_No        1     True
18                      DeviceProtection_No        1     True
15                          OnlineBackup_No        1     True
14                       OnlineSecurity_Yes        1     True
39                        TotalCharges_norm        1     True
12      

In [1045]:
# Save the features to a list to use in our modeling
true_values = result_RFE[result_RFE.Support == True]
feature_list = []
for g in true_values['Features']:
    feature_list.append(g)

# Create our X for the training and test sets using the recommended features.
RFE_X = X_train[feature_list]
RFE_Xtest = X_test[feature_list]

In [1046]:
#Create a testing function that will fit the test with PCA and our selected features from both RFECV and Random Forest
def testing_func(test):
    # This is a local list used to temporarily store scores
    scores=[]
    # This list serves the For loop below putting the variables in
    models = [RFE_X, y, RFE_Xtest, y_test]
    test.fit(models[0], models[1])
    print('Training Score: ', test.score(models[0], models[1]))
    print('Test Score: ', test.score(models[2], models[3]))
    print('Cross Validation Mean:', np.mean(cross_val_score(test, models[0], models[1], cv=5)))

    

In [1047]:
# SKlearn has a tool that will optimize MLP
start = datetime.now()
mlp = MLPClassifier()
# Set the hyperparameters
parameters = {'hidden_layer_sizes':[(400, 50, 25), (1000, 4)], 'alpha': 
              [0.03,0.05], 'learning_rate': ['adaptive'], 'activation': ['logistic']}
# Run it through the tool.
grid = GridSearchCV(mlp, parameters).fit(RFE_X, y)
predicted = grid.predict(RFE_Xtest)
print("The best parameters are {} with a score of %{}".format(grid.best_params_, grid.best_score_))

#Put those parameters in a dictionary to call.
parameters = grid.best_params_

print('It took the following time to complete this task:', datetime.now() - start)

The best parameters are {'activation': 'logistic', 'alpha': 0.05, 'hidden_layer_sizes': (400, 50, 25), 'learning_rate': 'adaptive'} with a score of %0.7615814696485623
It took the following time to complete this task: 0:00:16.807972


In [1048]:
start = datetime.now()
mlp = MLPClassifier(hidden_layer_sizes=parameters['hidden_layer_sizes'],
                   alpha=parameters['alpha'],
                   learning_rate=parameters['learning_rate'],
                   activation=parameters['activation'])
testing_func(mlp)

print('It took the following time to complete this task:', datetime.now() - start)

Training Score:  0.7619808306709265
Test Score:  0.7617504051863857
Cross Validation Mean: 0.5508
It took the following time to complete this task: 0:00:04.022871


In [1049]:
# Start with a simple sequential model
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(128, activation='sigmoid', input_shape=(RFE_Xtest.shape[1],)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(1, activation='sigmoid'))

model.summary()
sgd = SGD(lr = 0.1, momentum = 0.9, decay = 0, nesterov = False)
# Compile the model to put it all together.
model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_61 (Dense)             (None, 128)               2944      
_________________________________________________________________
dropout_66 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_62 (Dense)             (None, 128)               16512     
_________________________________________________________________
dropout_67 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_63 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_68 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_64 (Dense)             (None, 1)                 65        
Total para

In [1050]:
history = model.fit(RFE_X, y_train,
                    batch_size=64,
                    epochs=10,
                    verbose=1,
                    validation_data=(RFE_Xtest, y_test))
score = model.evaluate(RFE_Xtest, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 2504 samples, validate on 1234 samples
Epoch 1/10
2504/2504 [==============================] - 1s 509us/step - loss: 0.6710 - acc: 0.5723 - val_loss: 0.7238 - val_acc: 0.6532
Epoch 2/10
2504/2504 [==============================] - 0s 58us/step - loss: 0.5718 - acc: 0.7141 - val_loss: 0.5166 - val_acc: 0.7480
Epoch 3/10
2504/2504 [==============================] - 0s 50us/step - loss: 0.5391 - acc: 0.7344 - val_loss: 0.5384 - val_acc: 0.7164
Epoch 4/10
2504/2504 [==============================] - 0s 48us/step - loss: 0.5319 - acc: 0.7476 - val_loss: 0.5369 - val_acc: 0.7439
Epoch 5/10
2504/2504 [==============================] - 0s 49us/step - loss: 0.5427 - acc: 0.7404 - val_loss: 0.5104 - val_acc: 0.7585
Epoch 6/10
2504/2504 [==============================] - 0s 48us/step - loss: 0.5189 - acc: 0.7476 - val_loss: 0.5289 - val_acc: 0.7528
Epoch 7/10
2504/2504 [==============================] - 0s 49us/step - loss: 0.5204 - acc: 0.7544 - val_loss: 0.5159 - val_acc: 0.7423
Epoch 

In [1051]:
x_train = RFE_X.values
x_test = RFE_Xtest.values

In [1052]:

# set parameters:
max_features = 20
maxlen = 16
batch_size = 64
embedding_dims = 25
filters = 216
kernel_size = 3
hidden_dims = 256
epochs = 100

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Pad sequences (samples x time)
x_train shape: (2504, 16)
x_test shape: (1234, 16)


In [1053]:
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.1))
model.add(Activation('sigmoid'))
# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.1))
model.add(Activation('relu'))
# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 2504 samples, validate on 1234 samples
Epoch 1/100
2504/2504 [==============================] - 2s 723us/step - loss: 0.7223 - acc: 0.4948 - val_loss: 0.6940 - val_acc: 0.4838
Epoch 2/100
2504/2504 [==============================] - 1s 200us/step - loss: 0.6807 - acc: 0.5667 - val_loss: 0.6508 - val_acc: 0.6199
Epoch 3/100
2504/2504 [==============================] - 1s 206us/step - loss: 0.6460 - acc: 0.6326 - val_loss: 0.6280 - val_acc: 0.6588
Epoch 4/100
2504/2504 [==============================] - 1s 205us/step - loss: 0.6531 - acc: 0.6342 - val_loss: 0.6276 - val_acc: 0.6588
Epoch 5/100
2504/2504 [==============================] - 1s 207us/step - loss: 0.6462 - acc: 0.6382 - val_loss: 0.6308 - val_acc: 0.6507
Epoch 6/100
2504/2504 [==============================] - 1s 224us/step - loss: 0.6461 - acc: 0.6334 - val_loss: 0.6271 - val_acc: 0.6588
Epoch 7/100
2504/2504 [==============================] - 1s 214us/step - loss: 0.6484 - acc: 0.6358 - val_loss: 0.6385 - val_acc: 

2504/2504 [==============================] - 1s 229us/step - loss: 0.6237 - acc: 0.6518 - val_loss: 0.6240 - val_acc: 0.6677
Epoch 61/100
2504/2504 [==============================] - 1s 219us/step - loss: 0.6281 - acc: 0.6542 - val_loss: 0.6235 - val_acc: 0.6677
Epoch 62/100
2504/2504 [==============================] - 1s 238us/step - loss: 0.6330 - acc: 0.6466 - val_loss: 0.6224 - val_acc: 0.6677
Epoch 63/100
2504/2504 [==============================] - 1s 231us/step - loss: 0.6334 - acc: 0.6454 - val_loss: 0.6234 - val_acc: 0.6677
Epoch 64/100
2504/2504 [==============================] - 1s 213us/step - loss: 0.6311 - acc: 0.6538 - val_loss: 0.6359 - val_acc: 0.6345
Epoch 65/100
2504/2504 [==============================] - 1s 209us/step - loss: 0.6312 - acc: 0.6450 - val_loss: 0.6220 - val_acc: 0.6677
Epoch 66/100
2504/2504 [==============================] - 1s 229us/step - loss: 0.6267 - acc: 0.6542 - val_loss: 0.6218 - val_acc: 0.6677
Epoch 67/100
2504/2504 [=======================

In [1054]:
print('It took the following time to complete this notebook:', datetime.now() - main_start)

It took the following time to complete this notebook: 0:01:23.995665
